# Exp 002 
- 모든 프로파일 단체 학습

In [1]:
import os
import numpy as np
import tensorflow as tf

from seq import *
from nets.snet import *
np.set_printoptions(suppress=True)

In [2]:
def init_random_seed(seed):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
init_random_seed(1234)

In [3]:
def set_gpu_memory_growth_mode(gpu_id=0):
    import tensorflow as tf
    # tf.config.gpu.set_per_process_memory_growth(True) --> TF 1.x
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if not gpus:
        return
    try:
        tf.config.experimental.set_memory_growth(gpus[gpu_id], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
        
set_gpu_memory_growth_mode()

In [48]:
def make_loss_fn():
    @tf.function
    def gaze_loss(label, pred):
        return tf.reduce_mean(tf.losses.mse(label, pred))
    def loss(truth, pred):
        r_t, r_p = truth[:,0], pred[:0]
        r_txy, r_pxy = truth[:,1:], pred[:,1:]
        dist_diff = r_p - r_t
        angle_diff = tf.reduce_mean(tf.losses.mse(r_txy, r_pxy))
        l2_norm = 10*tf.abs(1 - (tf.math.square(y_pred[0]) + tf.math.square(y_pred[1])))
        return angle_diff + l2_norm
    # return loss  
    return gaze_loss
 
def get_mean_distance_metric():
    def mean_distance(y_true, y_pred):
        square = tf.math.square(y_pred - y_true)
        reduce_sum = tf.math.reduce_sum(square, axis=1)
        dists = tf.math.sqrt(reduce_sum)
        return tf.math.reduce_mean(dists)
    def debug_metric(y_true, y_pred):
        r_t, r_p = y_true[:,0], y_pred[:0]
        theta_t, theta_p = y_true[:,1:], y_pred[:,1:]
        dist_diff = r_p - r_t
        angle_diff = tf.reduce_min(tf.abs(theta_p - theta_t))
        return angle_diff
    # return debug_metric 
    return mean_distance
    
def make_model():
    net = SNet.create()
    l = make_loss_fn()
    # lr = tf.keras.experimental.CosineDecayRestarts(0.01, 10)
    o = tf.keras.optimizers.Adam(learning_rate=0.001)
    m = [get_mean_distance_metric()]
    net.compile(loss=l, optimizer=o, metrics=m)
    return net

def lr_schedule(epoch):
        learning_rate = 0.01
        if epoch > 10:
            learning_rate = 0.005
        if epoch > 20:
            learning_rate = 0.001
        if epoch > 50:
            learning_rate = 0.0008
        if epoch > 80:
            learning_rate = 0.00001
        tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
        return learning_rate
    
def get_callbacks():
    lr = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
    return []

# 데이터셋 

In [41]:
tf_root_path = "/home/elvin/banner/mnt/ssd3/nps/tf"

In [42]:
train_tf_paths = grep_files(tf_root_path, 'tr-00000.tf')
valid_tf_paths = grep_files(tf_root_path, 'val-00000.tf')
train_ds = tf.data.TFRecordDataset(train_tf_paths)
valid_ds = tf.data.TFRecordDataset(valid_tf_paths)

In [43]:
def decode(serialized_item):
    parsed = tf.io.parse_example(
        serialized_item,
        features={
            "face_l"  : tf.io.VarLenFeature(dtype=tf.string),
            "face_r"  : tf.io.VarLenFeature(dtype=tf.string),
            "support" : tf.io.FixedLenFeature([56], dtype=tf.float32),
            "target_xy" : tf.io.FixedLenFeature([2], dtype=tf.float32)
        }
    )
    
    face_l = parsed['face_l'].values[0]
    face_r = parsed['face_r'].values[0]
    support = parsed['support']
    t_xy = parsed['target_xy']
    
    r = tf.norm(t_xy, axis=-1)
    theta = tf.math.atan2(t_xy[1], t_xy[0])
    r_x, r_y = tf.math.cos(theta), tf.math.sin(theta)
    # label = tf.stack([r, r_x, r_y])
    label = tf.stack([r_x, r_y])
    
    face_l = tf.io.decode_jpeg(face_l)
    face_r = tf.io.decode_jpeg(face_r)
    face_l = tf.image.convert_image_dtype(face_l, tf.float32)
    face_r = tf.image.convert_image_dtype(face_r, tf.float32)
    
    return (face_l, face_r, support), label

In [44]:
train_seq = train_ds.map(decode, num_parallel_calls=16).prefetch(tf.data.experimental.AUTOTUNE).batch(16)
valid_seq = valid_ds.map(decode, num_parallel_calls=16).prefetch(tf.data.experimental.AUTOTUNE).batch(64)

In [49]:
model = make_model()

In [50]:
model.fit(train_seq, validation_data=valid_seq, epochs=1, verbose=1)

2614/2614 [==============================] - 100s 38ms/step - loss: 0.1143 - mean_distance: 0.3924 - val_loss: 0.2735 - val_mean_distance: 0.6375


In [11]:
# ts = valid_seq.take(10)
# for t in ts:
#     for l in t[1].numpy():
#         r, theta = l[0], l[1]
#         print(r, np.rad2deg(theta))

In [51]:
test_sample = valid_seq.take(10)
r = model.predict(test_sample)
labels = []
for x in test_sample:
    for label in x[1].numpy():
        labels.append(label)
        
angle_diffs = []
for i in range(len(labels)):
    t, p = labels[i], r[i]
    # d_t, a_t = np.linalg.norm(t), np.arctan2(t[0],t[1])
    # d_p, a_p = np.linalg.norm(p), np.arctan2(p[0],p[1])
    
    t_x, t_y = np.around(t, decimals=3)
    p_x, p_y = np.around(p, decimals=3)
    lt, lp = np.sqrt(t_x**2 + t_y**2), np.sqrt(p_x**2+p_y**2)
    t_a, p_a = np.arctan2(t_y, t_x)*360/(2*np.pi), np.arctan2(p_y, p_x)*360/(2*np.pi)
  
    d = np.around(np.abs(t_a - p_a), decimals=2)
    print(np.around(lt, decimals=1), "{:5s}".format(''), np.around(lp, decimals=2), "{:5s}".format(''), d)
    # print(t, p)
    # print(lt, lp)
    # angle_diffs.append(d)
    
# print(angle_diffs)

1.0       0.14       91.28
1.0       0.97       7.01
1.0       0.07       4.52
1.0       0.7       249.67
1.0       0.2       68.83
1.0       0.13       7.15
1.0       0.79       95.24
1.0       1.35       350.39
1.0       1.09       349.08
1.0       0.07       18.83
1.0       1.04       30.65
1.0       0.39       24.18
1.0       0.41       22.85
1.0       0.48       19.21
1.0       0.38       59.22
1.0       1.3       328.78
1.0       0.82       31.85
1.0       0.25       9.39
1.0       0.61       2.83
1.0       1.31       325.76
1.0       0.83       26.39
1.0       0.92       21.91
1.0       0.88       76.07
1.0       0.28       37.64
1.0       0.42       7.17
1.0       1.01       78.21
1.0       0.48       8.74
1.0       0.3       187.55
1.0       1.16       317.52
1.0       0.34       32.55
1.0       0.65       25.43
1.0       0.69       8.37
1.0       0.67       61.19
1.0       0.76       69.02
1.0       0.55       47.16
1.0       1.04       242.4
1.0       0.64       17.18
1.0   

In [39]:
test_sample = valid_seq.take(10)
r = model.predict(test_sample)
labels = []
for x in test_sample:
    for label in x[1].numpy():
        labels.append(label)
        
angle_diffs = []
for i in range(len(labels)):
    t, p = labels[i], r[i]
    # d_t, a_t = np.linalg.norm(t), np.arctan2(t[0],t[1])
    # d_p, a_p = np.linalg.norm(p), np.arctan2(p[0],p[1])
    
    t_x, t_y = np.around(t, decimals=3)
    p_x, p_y = np.around(p, decimals=3)
    lt, lp = np.sqrt(t_x**2 + t_y**2), np.sqrt(p_x**2+p_y**2)
    t_a, p_a = np.arctan2(t_y, t_x)*360/(2*np.pi), np.arctan2(p_y, p_x)*360/(2*np.pi)
  
    d = np.around(np.abs(t_a - p_a), decimals=2)
    print(np.around(lt, decimals=1), "{:5s}".format(''), np.around(lp, decimals=2), "{:5s}".format(''), d)
    # print(t, p)
    # print(lt, lp)
    # angle_diffs.append(d)
    
# print(angle_diffs)

1.0       0.84       6.21
1.0       0.53       65.86
1.0       1.02       9.25
1.0       0.39       1.08
1.0       0.51       30.11
1.0       0.93       2.57
1.0       0.59       0.31
1.0       1.15       22.04
1.0       0.6       1.96
1.0       0.71       30.19
1.0       0.55       30.82
1.0       0.97       5.13
1.0       1.03       7.8
1.0       0.99       15.12
1.0       0.39       16.93
1.0       1.17       42.28
1.0       0.78       21.57
1.0       1.15       5.34
1.0       0.55       15.98
1.0       1.06       36.7
1.0       0.59       4.84
1.0       1.1       13.67
1.0       0.58       34.38
1.0       0.84       50.82
1.0       0.61       28.35
1.0       0.49       22.52
1.0       1.13       7.07
1.0       0.69       66.73
1.0       0.97       45.03
1.0       1.03       12.92
1.0       0.61       2.4
1.0       0.59       16.87
1.0       0.5       24.46
1.0       0.47       8.69
1.0       0.45       37.79
1.0       0.27       33.41
1.0       1.03       4.86
1.0       0.62       